In [2]:
import requests
import datetime

import pandas as pd

In [3]:
API_KEY = "bdnTOY3Wx7hYKcG7xjRo5ALgdcEYGiLv"

In [4]:
url="https://api.nytimes.com/svc/books/v3/lists"
date = "current" #str(datetime.datetime.today().date() - datetime.timedelta(days=1))
query_url = url + "/" +"names.json?"+"api-key=" + API_KEY

r = requests.get(query_url)

data = r.json()

In [5]:
query_url

'https://api.nytimes.com/svc/books/v3/lists/names.json?api-key=bdnTOY3Wx7hYKcG7xjRo5ALgdcEYGiLv'

- After that, you need to write a script to retrieve lists according to the following requirements.:
    1. Use the API endpoint /lists/names.json to find the 5 newest monthly lists.
    2. Output list_name_encoded and oldest_published_date for all 5 lists in a single JSON file. You can choose the JSON field names and format you want.

- Finally, you need a script to retrieve books:
    1. Must accept as an argument the path to the JSON file above, and optionally an end_date
    2. Iterate over the 5 lists. For each list: 
        - Use the API endpoint to retrieve the books in that list for that date
        - Logic for iteration on dates is: 
            - first request is on oldest_published_date (from your file), 
            - then follow next_published_date that comes in the response, until there is no more data available or next_published_date is after the end_date parameter.
        - Please respect the pagination (fetch data from all pages, not only the first).
        - Output must be: a single NDJson (newline delimited json) file (per list) containing the entire API response for all months in that list, one response per row. 

In [168]:
def part1(API_KEY=API_KEY) -> dict:
    
    
    url = "https://api.nytimes.com/svc/books/v3/lists/names.json"
    
    params = {
        "api-key": f"{API_KEY}"
    }

    response = requests.get(url, params=params)
    

    data = response.json()
    
    # Filter for monthly lists
    data = [data for data in data["results"] if data["updated"]=="MONTHLY"]
    
    # Sort for finding the newest 4 lists
    data = sorted(data, key=lambda data: data['oldest_published_date'])[:5]

    output_data = [{key:value for key,value in d.items() 
               if key in ['list_name_encoded', 'oldest_published_date']} 
              for d in data]
    
    # Write output to a JSON file
    with open("newest_monthly_lists.json", "w") as f:
        json.dump(output_data, f, indent=4)
    
    return output_data

In [155]:
import os
import requests
import json

def retrieve_books(path_to_file, end_date=None):
    # Read the input data from the JSON file
    with open(path_to_file, 'r') as f:
        data = json.load(f)

    # Loop through each monthly list and retrieve the books for each month
    for lst in data:
        list_name_encoded = lst['list_name_encoded']
        oldest_published_date = lst['oldest_published_date']

        # Create a list to hold the API responses for each month
        responses = []

        # Initialize the API endpoint URL and parameters
        url = f'https://api.nytimes.com/svc/books/v3/lists/{list_name_encoded}.json'
        params = {
            'api-key': f'{API_KEY}',
            'published_date': oldest_published_date
        }

        # Loop until there is no more data available or we reach the end date
        while True:
            # Send a GET request to the API endpoint with the current parameters
            response = requests.get(url, params=params)

            # Check if the response was successful
            if response.status_code == 200:
                # Parse the response data as JSON
                data = json.loads(response.text)

                # Append the response data to the list of responses
                responses.append(data)

                # Check if there is more data available
                if 'next_published_date' in data and (not end_date or data['next_published_date'] <= end_date):
                    # Update the parameters with the next_published_date
                    params['published_date'] = data['next_published_date']
                else:
                    break
            else:
                print(f'Error: {response.status_code} - {response.reason}')
                break

        # Write the list of responses to a NDJSON file
        output_filename = f'{list_name_encoded}.ndjson'
        with open(output_filename, 'w') as f:
            for response in responses:
                f.write(json.dumps(response) + '\n')

        print(f'Output data saved to {os.path.abspath(output_filename)}')


In [169]:
part1()

[{'list_name_encoded': 'hardcover-business-books',
  'oldest_published_date': '2011-07-03'},
 {'list_name_encoded': 'paperback-business-books',
  'oldest_published_date': '2011-07-03'},
 {'list_name_encoded': 'hardcover-political-books',
  'oldest_published_date': '2011-07-03'},
 {'list_name_encoded': 'science', 'oldest_published_date': '2013-04-14'},
 {'list_name_encoded': 'food-and-fitness',
  'oldest_published_date': '2013-09-01'}]

In [156]:
retrieve_books("newest_monthly_lists.json")

Output data saved to /Users/natitaw/Documents/GitHub/nytimes-books/exp/hardcover-business-books.ndjson
Output data saved to /Users/natitaw/Documents/GitHub/nytimes-books/exp/paperback-business-books.ndjson
Output data saved to /Users/natitaw/Documents/GitHub/nytimes-books/exp/hardcover-political-books.ndjson
Output data saved to /Users/natitaw/Documents/GitHub/nytimes-books/exp/science.ndjson
Output data saved to /Users/natitaw/Documents/GitHub/nytimes-books/exp/food-and-fitness.ndjson


In [181]:
# Read the input data from the JSON file
with open("newest_monthly_lists.json", 'r') as f:
    data = json.load(f)


In [182]:
data[0]

{'list_name_encoded': 'hardcover-business-books',
 'oldest_published_date': '2011-07-03'}

In [183]:
lst = data[0]
# Loop through each monthly list and retrieve the books for each month

list_name_encoded = lst['list_name_encoded']
oldest_published_date = lst['oldest_published_date']

# Create a list to hold the API responses for each month
responses = []

# Initialize the API endpoint URL and parameters
url = f'https://api.nytimes.com/svc/books/v3/lists/{list_name_encoded}.json'
params = {
    'api-key': f'{API_KEY}',
    'published_date': oldest_published_date
}


In [184]:
response = requests.get(url, params=params)

In [186]:
pd.DataFrame(json.loads(response.text))

,status,copyright,num_results,last_modified,results
bestsellers_date,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,2013-09-28
books,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,"[{'rank': 1, 'rank_last_week': 0, 'weeks_on_li..."
corrections,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,[]
display_name,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,Hardcover Business Books
list_name,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,Hardcover Business Books
list_name_encoded,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,hardcover-business-books
next_published_date,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,
normal_list_ends_at,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,15
previous_published_date,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,2013-09-01
published_date,OK,Copyright (c) 2023 The New York Times Company....,15,2019-08-29T21:23:18-04:00,2013-10-13


In [179]:

# Loop until there is no more data available or we reach the end date
while True:
    # Send a GET request to the API endpoint with the current parameters
    response = requests.get(url, params=params)

    # Check if the response was successful
    if response.status_code == 200:
        # Parse the response data as JSON
        data = json.loads(response.text)

        # Append the response data to the list of responses
        responses.append(data)

        # Check if there is more data available
        if 'next_published_date' in data and (not end_date or data['next_published_date'] <= end_date):
            # Update the parameters with the next_published_date
            params['published_date'] = data['next_published_date']
        else:
            break
    else:
        print(f'Error: {response.status_code} - {response.reason}')
        break

# Write the list of responses to a NDJSON file
output_filename = f'{list_name_encoded}.ndjson'
with open(output_filename, 'w') as f:
    for response in responses:
        f.write(json.dumps(response) + '\n')

print(f'Output data saved to {os.path.abspath(output_filename)}')

Output data saved to /Users/natitaw/Documents/GitHub/nytimes-books/exp/hardcover-business-books.ndjson


In [187]:
with open("science.ndjson", 'r') as f:
    data = json.load(f)

In [189]:
pd.DataFrame(data)

,status,copyright,num_results,last_modified,results
bestsellers_date,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,2019-08-31
books,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,"[{'rank': 1, 'rank_last_week': 1, 'weeks_on_li..."
corrections,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,[]
display_name,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,Science
list_name,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,Science
list_name_encoded,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,science
next_published_date,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,
normal_list_ends_at,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,10
previous_published_date,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,2019-08-01
published_date,OK,Copyright (c) 2023 The New York Times Company....,10,2019-09-05T11:38:01-04:00,2019-09-15


In [195]:
pd.DataFrame(data['results']['books'])

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,...,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,1,0,0,0,0062316117,9780062316110,Harper Perennial,,0.00,...,192,http://www.amazon.com/Sapiens-A-Brief-History-...,,,,,,"[{'isbn10': '0062316095', 'isbn13': '978006231...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/8396e181-cd69-5074-a009-ec8f051c54f2
1,2,2,0,0,0,0143127748,9780143127741,Penguin,,0.00,...,193,http://www.amazon.com/The-Body-Keeps-Score-Hea...,,https://www.nytimes.com/2018/10/18/books/revie...,,,,"[{'isbn10': '0670785938', 'isbn13': '978067078...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e3e33e9d-0e67-5fec-b0d2-2ecddc38ce0e
2,3,4,0,0,0,1250076226,9781250076229,Picador,,0.00,...,495,http://www.amazon.com/Being-Mortal-Medicine-Wh...,,https://www.nytimes.com/2014/10/17/arts/being-...,,https://www.nytimes.com/2014/11/09/books/revie...,,"[{'isbn10': '0805095152', 'isbn13': '978080509...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/2dae1f76-020a-586b-8fa3-48f8f808d595
3,4,6,0,0,0,0374533555,9780374533557,"Farrar, Straus & Giroux",,0.00,...,490,http://www.amazon.com/Thinking-Fast-Slow-Danie...,,https://www.nytimes.com/2011/11/27/books/revie...,,https://www.nytimes.com/2011/11/27/books/revie...,,"[{'isbn10': '0374275637', 'isbn13': '978037427...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/1e1ffccd-d526-5154-84de-1628b31454e9
4,5,0,0,0,0,1524743410,9781524743413,Dutton,,0.00,...,495,https://www.amazon.com/Mosquito-Human-History-...,,,,,,"[{'isbn10': '1524743410', 'isbn13': '978152474...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/cf7352fd-374f-5c6b-8f3d-daca3dc57993
5,6,7,0,0,0,1400052181,9781400052189,Broadway,,0.00,...,280,http://www.amazon.com/The-Immortal-Life-Henrie...,,https://www.nytimes.com/2010/02/03/books/03boo...,https://www.nytimes.com/2010/03/03/books/excer...,https://www.nytimes.com/2010/02/07/books/revie...,https://www.nytimes.com/2010/02/02/health/02se...,"[{'isbn10': '1400052173', 'isbn13': '978140005...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/eaa8807f-166e-539f-9b4b-029048dca430
6,7,0,0,0,0,0316551309,9780316551304,"Little, Brown",,0.00,...,495,https://www.amazon.com/Dopesick-Dealers-Doctor...,,https://www.nytimes.com/2018/07/31/books/revie...,,,,"[{'isbn10': '0316551244', 'isbn13': '978031655...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/6ed4d478-5408-513e-82e9-7e7ca1cd519c
7,8,8,0,0,0,1501144324,9781501144325,Scribner,,0.00,...,495,https://www.amazon.com/Why-We-Sleep-Unlocking-...,,,,,,"[{'isbn10': '1501144316', 'isbn13': '978150114...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/6f7c99ee-b6e0-50d5-9708-5b7a008d4332
8,9,0,0,0,0,0525512195,9780525512196,Spiegel & Grau,,0.00,...,495,https://www.amazon.com/Lessons-21st-Century-Yu...,,https://www.nytimes.com/2018/09/04/books/revie...,,,,"[{'isbn10': '0525512179', 'isbn13': '978052551...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/3a44419d-5e0d-52dd-9138-8b9a0a7b8f36
9,10,10,0,0,0,0307352153,9780307352156,Broadway,,0.00,...,495,http://www.amazon.com/Quiet-Power-Introverts-W...,,,,https://www.nytimes.com/2012/02/12/books/revie...,,"[{'isbn10': '0307352145', 'isbn13': '978030735...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e2d80da4-6b71-57d0-b2a1-4a3d3f22a73b


In [192]:
data

{'status': 'OK',
 'copyright': 'Copyright (c) 2023 The New York Times Company.  All Rights Reserved.',
 'num_results': 10,
 'last_modified': '2019-09-05T11:38:01-04:00',
 'results': {'list_name': 'Science',
  'list_name_encoded': 'science',
  'bestsellers_date': '2019-08-31',
  'published_date': '2019-09-15',
  'published_date_description': 'latest',
  'next_published_date': '',
  'previous_published_date': '2019-08-01',
  'display_name': 'Science',
  'normal_list_ends_at': 10,
  'updated': 'MONTHLY',
  'books': [{'rank': 1,
    'rank_last_week': 1,
    'weeks_on_list': 0,
    'asterisk': 0,
    'dagger': 0,
    'primary_isbn10': '0062316117',
    'primary_isbn13': '9780062316110',
    'publisher': 'Harper Perennial',
    'description': '',
    'price': '0.00',
    'title': 'SAPIENS',
    'author': 'Yuval Noah Harari',
    'contributor': 'by Yuval Noah Harari',
    'contributor_note': '',
    'book_image': 'https://storage.googleapis.com/du-prd/books/images/9780062316097.jpg',
    'boo